# PreSumm Sandbox

## Setup

Download the models and install them in the `models` directory similar to what was described in the "PreSummTextSummarizationwithPretrainedEncoders_Inference" notebook:
```
!gdown https://drive.google.com/uc?id=1kKWoV0QCbeIuFt85beQgJ4v0lujaXobJ&export=download #CNN/DM Extractive bertext_cnndm_transformer.pt
!gdown https://drive.google.com/uc?id=1-IKVCtc4Q-BdZpjXc4s70_fRsWnjtYLr&export=download #CNN/DM Abstractive model_step_148000.pt 
!gdown https://drive.google.com/uc?id=1H50fClyTkNprWJNh10HWdGEdDdQIkzsI&export=download #XSUM (One Sentence Summary) model_step_30000.pt   
!unzip /content/PreSumm/models/bertext_cnndm_transformer.zip
!unzip /content/PreSumm/models/bertsumextabs_cnndm_final_model.zip
!unzip /content/PreSumm/models/bertsumextabs_xsum_final_model.zip
!mkdir /content/PreSumm/models/CNN_DailyMail_Extractive
!mkdir /content/PreSumm/models/CNN_DailyMail_Abstractive
!mkdir /content/PreSumm/models/XSUM_OneSentence
!mv /content/PreSumm/models/bertext_cnndm_transformer.pt /content/PreSumm/models/CNN_DailyMail_Extractive
!mv /content/PreSumm/models/model_step_148000.pt /content/PreSumm/models/CNN_DailyMail_Abstractive
!mv /content/PreSumm/models/model_step_30000.pt /content/PreSumm/models/XSUM_OneSentence
```

Create a new anaconda environment from the environment.yml file, activate the environment and run jupyter:
```
$ conda env create --file environment.yml
$ conda activate presumm
$ jupyter notebook .
```

In [1]:
from IPython.core.display import display as core_display, HTML
core_display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
%cd src

/Users/jim.crotinger/git/github/nlp/PreSumm/src


In [3]:
!ls

__pycache__          distributed.py       post_stats.py        train.py             translate
cal_rouge.py         models               prepro               train_abstractive.py
default.profraw      others               preprocess.py        train_extractive.py


## Arguments to `train.py`

In [4]:
!python train.py -h

usage: train.py [-h] [-task {ext,abs}] [-encoder {bert,baseline}] [-mode {train,validate,test,test_text}]
                [-bert_data_path BERT_DATA_PATH] [-model_path MODEL_PATH] [-result_path RESULT_PATH] [-temp_dir TEMP_DIR]
                [-text_src TEXT_SRC] [-text_tgt TEXT_TGT] [-batch_size BATCH_SIZE] [-test_batch_size TEST_BATCH_SIZE]
                [-max_ndocs_in_batch MAX_NDOCS_IN_BATCH] [-max_pos MAX_POS] [-use_interval [USE_INTERVAL]] [-large [LARGE]]
                [-load_from_extractive LOAD_FROM_EXTRACTIVE] [-sep_optim [SEP_OPTIM]] [-lr_bert LR_BERT] [-lr_dec LR_DEC]
                [-use_bert_emb [USE_BERT_EMB]] [-share_emb [SHARE_EMB]] [-finetune_bert [FINETUNE_BERT]]
                [-dec_dropout DEC_DROPOUT] [-dec_layers DEC_LAYERS] [-dec_hidden_size DEC_HIDDEN_SIZE]
                [-dec_heads DEC_HEADS] [-dec_ff_size DEC_FF_SIZE] [-enc_hidden_size ENC_HIDDEN_SIZE]
                [-enc_ff_size ENC_FF_SIZE] [-enc_dropout ENC_DROPOUT] [-enc_layers ENC_LAYERS] [-ex

## Run Abstractive Summary on test file

In [5]:
mode = "test_text"
task = "abs"
test_from = "../models/CNN_DailyMail_Abstractive/model_step_148000.pt"

# I've added a couple of stories to the test stories that are checked in. You can change these to just use `temp.raw_src` instead.

text_source = "../raw_data/test_stories"
result_path = "../results/test_stories"

log_file = "../logs/val_abs_bert_raw_test"
std_args = "-batch_size 32 -test_batch_size 500 -report_rouge False -sep_optim true -use_interval true " + \
        "-visible_gpus -1 -max_pos 512  -max_length 200 -alpha 0.95 -min_length 50" # -max_src_nsents 100

In [6]:
!echo train.py -task {task} -mode {mode} -text_src {text_source} -test_from {test_from} -log_file {log_file} -result_path {result_path} {std_args}

train.py -task abs -mode test_text -text_src ../raw_data/test_stories -test_from ../models/CNN_DailyMail_Abstractive/model_step_148000.pt -log_file ../logs/val_abs_bert_raw_test -result_path ../results/test_stories -batch_size 32 -test_batch_size 500 -report_rouge False -sep_optim true -use_interval true -visible_gpus -1 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50


In [7]:
!python train.py -task {task} -mode {mode} -text_src {text_source} -test_from {test_from} -log_file {log_file} -result_path {result_path} # {std_args}

[2020-04-24 09:15:22,939 INFO] Loading checkpoint from ../models/CNN_DailyMail_Abstractive/model_step_148000.pt
Namespace(accum_count=1, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='../bert_data_new/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_raw_test', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_ndocs_in_batch=6, max_pos=512, max_tgt_len=140, min_length=15, mode='test_text', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/test_stories', sav

In [8]:
results = !cat ../results/test_stories.-1.candidate
for line in results:
    print(line + '\n\n')

terry jones ' style of visual comedy inspired many comedians who followed him<q>it was on python that he honed his directing skills<q>he wrote a number of books and fronted tv documentaries on ancient and medieval history


shohreh bayat says she is frightened to return home after she was criticized online for not wearing the appropriate headscarf<q>currently the chief adjudicator at the women 's world chess championship held in russia and china


bitcoin priced in usd on mt. goxabove is gox price as of nearly9:30 pm (gmt+2) ) still above the previous low<q>but dipping below the 38.2- retracement level


officials viewed the emergency rate cut they approved earlier this week as part of an international push to cushion the economy as the coronavirus threatens global growth




In [9]:
mode = "test_text"
task = "abs"
test_from = "../models/CNN_DailyMail_TransformerAbs/cnndm_baseline_best.pt"

# I've added a couple of stories to the test stories that are checked in. You can change these to just use `temp.raw_src` instead.

text_source = "../raw_data/test_stories"
result_path = "../results/test_stories_bb"

log_file = "../logs/val_abs_bert_raw_test"
std_args = "-batch_size 32 -test_batch_size 500 -report_rouge False -sep_optim true -use_interval true " + \
        "-visible_gpus -1 -max_pos 512  -max_length 200 -alpha 0.95 -min_length 50" # -max_src_nsents 100

In [10]:
!python train.py -task {task} -mode {mode} -text_src {text_source} -test_from {test_from} -log_file {log_file} -result_path {result_path} # {std_args}

[2020-04-24 09:15:56,448 INFO] Loading checkpoint from ../models/CNN_DailyMail_TransformerAbs/cnndm_baseline_best.pt
Namespace(accum_count=1, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='../bert_data_new/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=512, dec_layers=6, enc_dropout=0.2, enc_ff_size=2048, enc_hidden_size=512, enc_layers=6, encoder='baseline', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_raw_test', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_ndocs_in_batch=6, max_pos=512, max_tgt_len=140, min_length=15, mode='test_text', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/test_sto

In [11]:
results = !cat ../results/test_stories.-1.candidate
for line in results:
    print(line + '\n\n')

terry jones ' style of visual comedy inspired many comedians who followed him<q>it was on python that he honed his directing skills<q>he wrote a number of books and fronted tv documentaries on ancient and medieval history


shohreh bayat says she is frightened to return home after she was criticized online for not wearing the appropriate headscarf<q>currently the chief adjudicator at the women 's world chess championship held in russia and china


bitcoin priced in usd on mt. goxabove is gox price as of nearly9:30 pm (gmt+2) ) still above the previous low<q>but dipping below the 38.2- retracement level


officials viewed the emergency rate cut they approved earlier this week as part of an international push to cushion the economy as the coronavirus threatens global growth




## Run Extractive Summary on test file

In [12]:
mode = "test_text"
task = "ext"
test_from = "../models/CNN_DailyMail_Extractive/bertext_cnndm_transformer.pt"

# Annotated test data - haven't done this for the new stories I added...

text_source = "../raw_data/temp_ext.raw_src"
result_path = "../results/temp_ext.raw"

log_file = "../logs/val_abs_bert_raw_test"
std_args = "-batch_size 32 -test_batch_size 500 -report_rouge False -sep_optim true -use_interval true " + \
        "-visible_gpus -1 -max_pos 512  -max_length 200 -alpha 0.95 -min_length 50" # -max_src_nsents 100

In [13]:
!python train.py -task {task} -mode {mode} -text_src {text_source} -test_from {test_from} -log_file {log_file} -result_path {result_path} # {std_args}

[2020-04-24 09:16:29,572 INFO] Loading checkpoint from ../models/CNN_DailyMail_Extractive/bertext_cnndm_transformer.pt
Namespace(accum_count=1, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='../bert_data_new/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_raw_test', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_ndocs_in_batch=6, max_pos=512, max_tgt_len=140, min_length=15, mode='test_text', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/temp_ext.ra

In [14]:
results = !cat ../results/temp_ext.raw_step-1.candidate

for line in results:
    print(line + '\n\n')

this Terry Jones had a love of the absurd that contributed much to the anarchic humour of Monty Python's Flying Circus.<q>His style of visual comedy, leavened with a touch of the surreal, inspired many comedians who followed him.<q>A keen historian, he wrote a number of books and fronted TV documentaries on ancient and medieval history.


(CNN) An Iranian chess referee says she is frightened to return home after she was criticized online for not wearing the appropriate headscarf during an international tournament.<q>Currently the chief adjudicator at the Women's World Chess Championship held in Russia and China, Shohreh Bayat says she fears arrest after a photograph of her was taken during the event and was then circulated online in Iran.<q>The headscarf, or the hijab, has been a mandatory part of women's dress in Iran since the 1979 Islamic revolution but, in recent years, some women have mounted opposition and staged protests about headwear rules.




## Run One-Sentence Summary on test file

In [15]:
mode = "test_text"
task = "abs"
test_from = "../models/XSUM_OneSentence/model_step_30000.pt"

# I've added a couple of stories to the test stories that are checked in. You can change these to just use `temp.raw_src` instead.

text_source = "../raw_data/test_stories"
result_path = "../results/test_stories.one_sent"

log_file = "../logs/val_abs_bert_raw_test"


In [16]:
!python train.py -task {task} -mode {mode} -text_src {text_source} -test_from {test_from} -log_file {log_file} -result_path {result_path} # {std_args}

[2020-04-24 09:16:36,661 INFO] Loading checkpoint from ../models/XSUM_OneSentence/model_step_30000.pt
Namespace(accum_count=1, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='../bert_data_new/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_raw_test', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_ndocs_in_batch=6, max_pos=512, max_tgt_len=140, min_length=15, mode='test_text', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/test_stories.one_sent', save

In [17]:
results = !cat ../results/test_stories.one_sent.-1.candidate
for line in results:
    print(line + '\n\n')

terry jones , who was born in the english channel in the early hours of sunday morning , was a welsh student who wanted to leave the country for the first time .


in the latest series of fights in iran , millions of people have been protesting against the so-called islamic state -lrb- is -rrb- .


the price of bitcoin and other exchanges has fallen below the new low level .


the head of the us federal reserve has given his first official statement since the us central bank cut interest rates last month .


